<a href="https://colab.research.google.com/github/kioko91/ML-Group-Final-Task/blob/main/RNN_model_for_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 1 : Finding and choosing a suitable real world dataset for the above task

In [1]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing import sequence
from keras.optimizers import *
from keras.datasets import imdb  # import the built-in imdb dataset which is in Keras https://keras.io/api/datasets/imdb/
from sklearn.feature_extraction.text import *
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression

# Step 2 : Preparing the dataset ready for training and testing

In [2]:
# Setting the vocabulary size
vocabulary_size = 5000

# Load in training and test data using a different convention asd show how many training samples and test samples that are there
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=vocabulary_size)
print("Loaded dataset with {} training samples, {} test samples".format(len(X_train), len(X_test)))

17473536/17464789 [==============================] - 0s 0us/step
Loaded dataset with 25000 training samples, 25000 test samples


In [3]:
# Inspect a sample review of the data and its label
print("--- Review ---")
print(X_train[7])
print("--- Label ---")
print(y_train[7])

--- Review ---
[1, 4, 2, 716, 4, 65, 7, 4, 689, 4367, 2, 2343, 4804, 2, 2, 2, 2, 2315, 2, 2, 2, 2, 4, 2, 628, 2, 37, 9, 150, 4, 2, 4069, 11, 2909, 4, 2, 847, 313, 6, 176, 2, 9, 2, 138, 9, 4434, 19, 4, 96, 183, 26, 4, 192, 15, 27, 2, 799, 2, 2, 588, 84, 11, 4, 3231, 152, 339, 2, 42, 4869, 2, 2, 345, 4804, 2, 142, 43, 218, 208, 54, 29, 853, 659, 46, 4, 882, 183, 80, 115, 30, 4, 172, 174, 10, 10, 1001, 398, 1001, 1055, 526, 34, 3717, 2, 2, 2, 17, 4, 2, 1094, 871, 64, 85, 22, 2030, 1109, 38, 230, 9, 4, 4324, 2, 251, 2, 1034, 195, 301, 14, 16, 31, 7, 4, 2, 8, 783, 2, 33, 4, 2945, 103, 465, 2, 42, 845, 45, 446, 11, 1895, 19, 184, 76, 32, 4, 2, 207, 110, 13, 197, 4, 2, 16, 601, 964, 2152, 595, 13, 258, 4, 1730, 66, 338, 55, 2, 4, 550, 728, 65, 1196, 8, 1839, 61, 1546, 42, 2, 61, 602, 120, 45, 2, 6, 320, 786, 99, 196, 2, 786, 2, 4, 225, 4, 373, 1009, 33, 4, 130, 63, 69, 72, 1104, 46, 1292, 225, 14, 66, 194, 2, 1703, 56, 8, 803, 1004, 6, 2, 155, 11, 4, 2, 3231, 45, 853, 2029, 8, 30, 6, 117, 430

In [4]:
# We map word IDs back to words since they are represented by numbers and view the sample review of the data and its label in words
word2id = imdb.get_word_index()
id2word = {i: word for word, i in word2id.items()}
print("--- Review (with words) ---")
print([id2word.get(i, " ") for i in X_train[7]])
print("--- Label ---")
print(y_train[7])

1654784/1641221 [==============================] - 0s 0us/step
--- Review (with words) ---
['the', 'of', 'and', 'local', 'of', 'their', 'br', 'of', 'attention', 'widow', 'and', 'captures', 'parties', 'and', 'and', 'and', 'and', 'excitement', 'and', 'and', 'and', 'and', 'of', 'and', 'english', 'and', 'like', 'it', 'years', 'of', 'and', 'unintentional', 'this', 'hitchcock', 'of', 'and', 'learn', 'everyone', 'is', 'quite', 'and', 'it', 'and', 'such', 'it', 'bonus', 'film', 'of', 'too', 'seems', 'he', 'of', 'enough', 'for', 'be', 'and', 'editing', 'and', 'and', 'please', 'great', 'this', 'of', 'shoots', 'thing', '3', 'and', "it's", 'mentioning', 'and', 'and', 'given', 'parties', 'and', 'back', 'out', 'interesting', 'times', 'no', 'all', 'average', 'talking', 'some', 'of', 'nor', 'seems', 'into', 'best', 'at', 'of', 'every', 'cast', 'i', 'i', 'inside', 'keep', 'inside', 'large', 'viewer', 'who', 'obscure', 'and', 'and', 'and', 'movie', 'of', 'and', 'entirely', "you've", 'see', 'because', 'y

In [5]:
# We set the maximum number of words per document for both the training and testing
max_words = 500

# set the pad sequences in X_train and X_test
X_train = sequence.pad_sequences(X_train, maxlen=max_words)
X_test = sequence.pad_sequences(X_test, maxlen=max_words)

# Step 3 : Building a deep model using Keras

In [6]:
#Designing our model
embedding_size = 32
model = Sequential()
model.add(Embedding(vocabulary_size, embedding_size, input_length=max_words))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))

print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 500, 32)           160000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               53200     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________
None


# Step 4 : We compile, tune and train our model

In [7]:
# We then compile our model, specifying a loss function, optimizer, and metrics.  
#We choose accuracy since it will give us better results compared to the other metrics which will be shown at the end.
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [8]:
# We then specify training parameters including batch size and number of epochs
batch_size = 64
num_epochs = 3
# We tuned the model by reducing the number of epochs and setting the batch size

# We then specify some training data for validation which will not be used in training
X_valid, y_valid = X_train[:batch_size], y_train[:batch_size]  # first batch_size samples
X_train2, y_train2 = X_train[batch_size:], y_train[batch_size:]  # rest for training

#We finally train our model
model.fit(X_train2, y_train2, validation_data=(X_valid, y_valid),batch_size=batch_size, epochs=num_epochs)

Epoch 1/3
390/390 [==============================] - 194s 491ms/step - loss: 0.4502 - accuracy: 0.7822 - val_loss: 0.2926 - val_accuracy: 0.8750
Epoch 2/3
390/390 [==============================] - 191s 489ms/step - loss: 0.3100 - accuracy: 0.8736 - val_loss: 0.2421 - val_accuracy: 0.9375
Epoch 3/3
390/390 [==============================] - 191s 489ms/step - loss: 0.2458 - accuracy: 0.9038 - val_loss: 0.2258 - val_accuracy: 0.9219


# Step 5: Test the trained model and comparing it with other metrics

In [9]:
#To get other metrics we will use logistic regression to fit the model1
model1 = LogisticRegression()
model1.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [10]:
#We then make prediction on the test set
pred = model1.predict(X_test)

In [11]:
#Later we calculate precision, f1 measure and recall
precision = format(precision_score(y_test, pred))
recall = format(recall_score(y_test, pred))
f1_measure=format(f1_score(y_test, pred))

In [12]:
 #We then evaluate our model on the test set
scores = model.evaluate(X_test, y_test, verbose=0)  # This returns loss and other metrics specified in model.compile() which was accuracy.
print("Test accuracy:", scores[1])  # scores[1] should correspond to accuracy if you passed in metrics=['accuracy']
#Scores for the precision,f1_measure and recall
print('Precision: ',precision)
print('Recall: ',recall)
print('f1_measure: ',f1_measure)

#We can clearly see accuracy is the best metric to be used in this model rather than the others

Test accuracy: 0.8726400136947632
Precision:  0.5060663868752384
Recall:  0.53056
f1_measure:  0.5180238234719781
